## Tratamento de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv("training_data.csv")
test_data = pd.read_csv("test_data.csv")

In [ ]:
data.info()

In [ ]:
data.describe()

Como as colunas city_name e avg_precipitation possuem apenas 1 valor, podemos retirá-las, já que não vão ter nenhuma influência na previsão

In [ ]:
data.nunique()

data2 = data.drop(['city_name', 'avg_precipitation'], axis=1)
test_data2 = test_data.drop(['city_name', 'avg_precipitation'], axis=1)

In [ ]:
data.head(30)

In [ ]:
data2.isnull().sum()

In [ ]:
sns.heatmap(data2.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
sns.heatmap(test_data2.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
i = 0
data_roads = data2.copy()

#print(data2['affected_roads']).head(30)
all_roads = {}
for road_list in data_roads['affected_roads']:
    if type(road_list) == type(float(i)):
        #print("Iteração ::", i)
        data_roads.at[i, 'affected_roads'] = ","
    else: 
        road_list = str(road_list)
        if road_list != ",":
            roads = road_list.split(",")
            #print(roads)
            dic = {}
            
            for road in roads:
                #print(road)
                if road not in dic and road != "":
                    #print(road)
                    dic[road] = 0
                if road not in all_roads and road != "":
                    all_roads[road] = 0
            #data2['affected_roads'][i] = str(list(dic.keys()))
            data_roads.at[i, 'affected_roads'] = dic.keys()
            #print(list(dic.keys()))
    i = i + 1
print("Nº de estradas :: ", len(all_roads))
print("Estradas :: ", all_roads.keys())


i = 0

test_data_roads = test_data2.copy()

#print(data2['affected_roads']).head(30)
all_roads_test = {}
for road_list in test_data_roads['affected_roads']:
    if type(road_list) == type(float(i)):
        #print("Iteração ::", i)
        test_data_roads.at[i, 'affected_roads'] = ","
    else: 
        road_list = str(road_list)
        if road_list != ",":
            roads = road_list.split(",")
            #print(roads)
            dic = {}
            
            for road in roads:
                #print(road)
                if road not in dic and road != "":
                    #print(road)
                    dic[road] = 0
                if road not in all_roads_test and road != "":
                    all_roads_test[road] = 0
            #data2['affected_roads'][i] = str(list(dic.keys()))
            test_data_roads.at[i, 'affected_roads'] = dic.keys()
            #print(list(dic.keys()))
    i = i + 1
print("Nº de estradas :: ", len(all_roads_test))
print("Estradas :: ", all_roads_test.keys())



In [ ]:
data_roads.head(30)

In [ ]:
all_roads_list = sorted(all_roads.keys())

for road in all_roads_list:
    data_roads[road] = 0

data_roads['others'] = 0

all_roads_test_list = sorted(all_roads.keys())
for road in all_roads_test_list:
    if road in all_roads_list:
        test_data_roads[road] = 0

test_data_roads['others'] = 0
#data2.head(30)

In [ ]:
test_data_roads.info()

In [ ]:
data_roads.info()

In [ ]:
i = 0
for reg in data_roads:
    list = data_roads.at[i, 'affected_roads']
    for road in list:
        if road != ",":
            data_roads.at[i, road] = 1
    i = i + 1


i = 0
for reg in test_data_roads:
    list = test_data_roads.at[i, 'affected_roads']
    for road in list:
        if road != ",":
            if road not in all_roads.keys():
                test_data_roads.at[i,'others'] = 1
            else:
                test_data_roads.at[i, road] = 1
    i = i + 1

In [ ]:
print(test_data_roads.columns)

In [ ]:
data3 = data_roads.dropna()
sns.heatmap(data_roads.isnull(), yticklabels=False, cbar=False, cmap='viridis')

Como os valores dos incidentes, a nossa label, são categóricos, estes não poderão ser submetidos à maioria dos modelos de ML disponíveis. Assim, teremos de convertê-los para valores numéricos

In [ ]:
replace_map = {'incidents' : {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High':4}}

data4 = data3.replace(replace_map)
data4.head()

Como podemos ver, agora o tipo do feature Incidents é numérico

In [ ]:
data4.info()

Agora vamos tratar das datas

In [ ]:
data4['record_date'].head()

In [ ]:
data4['record_date'] = pd.to_datetime(data4['record_date'], format = "%Y-%m-%d %H:%M", errors='coerce')
assert data4['record_date'].isnull().sum() == 0, 'missing record date'
data4['record_date'].head()

test_data_roads['record_date'] = pd.to_datetime(test_data_roads['record_date'], format = "%Y-%m-%d %H:%M", errors='coerce')

In [ ]:
data4['record_date_year'] = data4['record_date'].dt.year
data4['record_date_month'] = data4['record_date'].dt.month
data4['record_date_day'] = data4['record_date'].dt.day
data4['record_date_hour'] = data4['record_date'].dt.hour
data4['record_date_minute'] = data4['record_date'].dt.minute

test_data_roads['record_date_year'] = test_data_roads['record_date'].dt.year
test_data_roads['record_date_month'] = test_data_roads['record_date'].dt.month
test_data_roads['record_date_day'] = test_data_roads['record_date'].dt.day
test_data_roads['record_date_hour'] = test_data_roads['record_date'].dt.hour
test_data_roads['record_date_minute'] = test_data_roads['record_date'].dt.minute


data4.head()
test_data_roads.head()

In [ ]:
#data4.nunique()

Aqui podemos verificar que o ano dos incidentes, bem como o minuto, são features com um único valor, sendo então essa informação inútil.

In [ ]:
data5 = data4.drop(['record_date_minute','record_date_year','affected_roads'], axis = 1)
test_data3 = test_data_roads.drop(['record_date_minute','record_date_year','affected_roads'], axis = 1)
print(data5['magnitude_of_delay'].unique())
print(data5['luminosity'].unique())
print(data5['avg_rain'].unique())
data5.info()
test_data3.info()

A 'magnitude_of_delay' tem 3 valores possíveis: UNDEFINED, MAJOR e MODERATE <br>
A luminosidade tem 3 valores possíveis: LIGHT, LOW_LIGHT, DARK <br>
A 'avg_rain' tem 4 valores possíveis: Sem Chuva, chuva moderada, chuva fraca, chuva forte <br>
Assim, teremos de transformar estes valores em valores numéricos, fazendo label enconding


In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [ ]:
data6 = encode_and_bind(data5, 'magnitude_of_delay')
data6.drop('magnitude_of_delay',axis=1, inplace=True)

data7 = encode_and_bind(data6, 'luminosity')
data7.drop('luminosity',axis=1, inplace=True)

data8 = encode_and_bind(data7, 'avg_rain')
data8.drop('avg_rain', axis=1, inplace=True)

test_data4 = encode_and_bind(test_data3, 'magnitude_of_delay')
test_data4.drop('magnitude_of_delay',axis=1, inplace=True)

test_data5 = encode_and_bind(test_data4, 'luminosity')
test_data5.drop('luminosity',axis=1, inplace=True)

test_data6 = encode_and_bind(test_data5, 'avg_rain')
test_data6.drop('avg_rain',axis=1, inplace=True)

data_ohe = data8
data_ohe.head()

test_ohe = test_data6

Introduzir as estações do ano como uma feature

In [ ]:
data7 = data6
data7['season'] = 0
test_data5 = test_data4
test_data5['season'] = 0

for i in data7.index:
    if data7.at[i, 'record_date_month'] in [1, 2,3]:
        data7.at[i, 'season'] = 1 # Winter
    elif data7.at[i, 'record_date_month'] in [4, 5, 6]:
        data7.at[i, 'season'] = 2 # Spring
    elif data7.at[i, 'record_date_month'] in [7, 8, 9]:
        data7.at[i, 'season'] = 3 # Summer
    elif data7.at[i, 'record_date_month'] in [10, 11, 12]:
        data7.at[i, 'season'] = 4 # Fall

for i in test_data5.index:
    if test_data5.at[i, 'record_date_month'] in [1, 2,3]:
        test_data5.at[i, 'season'] = 1 # Winter
    elif test_data5.at[i, 'record_date_month'] in [4, 5, 6]:
        test_data5.at[i, 'season'] = 2 # Spring
    elif test_data5.at[i, 'record_date_month'] in [7, 8, 9]:
        test_data5.at[i, 'season'] = 3 # Summer
    elif test_data5.at[i, 'record_date_month'] in [10, 11, 12]:
        test_data5.at[i, 'season'] = 4 # Fall

Depois de feita toda a etapa de feature engineering, vamos agora construir um modelo.

In [ ]:
data8 = data_ohe.copy()
data8['date_offset'] = (data8['record_date'].dt.month*100 + data8['record_date'].dt.day - 320)%1300

data8['season'] = pd.cut(data8['date_offset'], [0, 300, 602, 900, 1300], 
                      labels = ['spring', 'summer', 'autumn', 'winter'], include_lowest=True)
data8.drop(['date_offset', 'record_date'], axis = 1, inplace=True)
replace_map = { 'season' : {'spring': 1, 'summer': 2, 'autumn': 3, 'winter' : 4}}
data9 = data8.replace(replace_map)
data9['season'] = data9['season'].astype(int)

data_season = data9


test_data6 = test_ohe.copy()
test_data6['date_offset'] = (test_data6['record_date'].dt.month*100 + test_data6['record_date'].dt.day - 320)%1300

test_data6['season'] = pd.cut(test_data6['date_offset'], [0, 300, 602, 900, 1300], 
                      labels = ['spring', 'summer', 'autumn', 'winter'], include_lowest=True)
test_data6.drop(['date_offset', 'record_date'], axis = 1, inplace=True)
replace_map = { 'season' : {'spring': 1, 'summer': 2, 'autumn': 3, 'winter' : 4}}
test_data6 = test_data6.replace(replace_map)
test_data6['season'] = test_data6['season'].astype(int)

test_season = test_data6

In [ ]:
df = data_season
df_test = test_data6

train_data_x = df.drop('incidents', axis=1)
train_label_y = df['incidents']
test_data_x = df_test
df.head()

In [ ]:
df.to_csv("dataset2.csv", header=None, index=None, sep=';', mode='a')

In [ ]:
df.head()

In [ ]:
corr_matrix = data9.corr()
sns.heatmap(corr_matrix, annot=False)
plt.show()

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=2022)

In [ ]:
clf.fit(X, y)

In [ ]:
predictions_dt = clf.predict(X_test)
predictions_dt

In [ ]:
#replace_map = {'incidents' : {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High':4}}

outcome = []
for i in range(len(predictions_dt)):
    if predictions_dt[i] == 0:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'None'})
    elif predictions_dt[i] == 1:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Low'})
    elif predictions_dt[i] == 2:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Medium'})
    elif predictions_dt[i] == 3:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'High'})
    elif predictions_dt[i] == 4:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Very_High'})
    
print(outcome)
od = pd.DataFrame(outcome)
od.to_csv("out_file.csv", index=False,header=True)

## Random Forest

In [ ]:
#df = data9

from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [200, 250]
max_features = ['auto', 'sqrt',None]
max_depth = [10,15,18]
min_samples_split = [1,2]
min_samples_leaf = [1,2]
bootstrap = [True, False]
criterion = ['entropy']


param_grid = {'n_estimators' : n_estimators, 
              'max_features' : max_features,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf,
              'bootstrap' : bootstrap,
              'criterion' : criterion}
print(param_grid)

In [ ]:
rf = RandomForestClassifier()
#rf.fit(X_train, y_train)

#predictions = rf.predict(X_Test)
#predictions

#from sklearn.metrics import accuracy_score
#rf.score(X_train, y_train)



In [ ]:
from sklearn.model_selection import GridSearchCV
rf_GridSearch = GridSearchCV(estimator = rf , param_grid = param_grid, cv = 10, verbose = 2 , n_jobs = 4)

In [ ]:
rf_GridSearch = rf_GridSearch.fit(train_data_x, train_label_y)

In [ ]:
rf_GridSearch.best_params_

In [ ]:
predictions = rf_GridSearch.predict(test_data_x)
predictions

In [ ]:
predictions = rf_GridSearch.predict(X_train)
predictions

In [ ]:
print (f'Train Accuracy - : {rf_GridSearch.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_GridSearch.score(X_Test,y_test):.3f}')#

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
outcome = []
for i in range(len(predictions)):
    if predictions[i] == 0:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'None'})
    elif predictions[i] == 1:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Low'})
    elif predictions[i] == 2:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Medium'})
    elif predictions[i] == 3:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'High'})
    elif predictions[i] == 4:
        outcome.append({'RowId' : i+1 , 'Incidents' : 'Very_High'})
    
print(outcome)
od = pd.DataFrame(outcome)
od.to_csv("random_forest_grid_search_out.csv", index=False,header=True)